# Projet 6 Classifiez automatiquement des biens de consommation

## Prétraitement des données

### Import des librairies


In [60]:
import pandas as pd
import nltk
import matplotlib as plt
import numpy as np
import re
import ast

### Prétraitement des données de texte

Avant de faire traitement de nos texte, nous allons d'abord analyser notre jeu de données pour faire un premier nettoyage et avoir une connaissances de notre dataframe pour sélectionner les informations pertinantes à garder.Cette première exploration de la données va nous permettre de verifier la qualité de nos données et ainsi de faire les traitement nécessaires.

#### Import des données textuels

In [61]:
df_product = pd.read_csv(
    'dataset/Flipkart/flipkart_com-ecommerce_sample_1050.csv')
df_product.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa..."


#### Analyse exploratoire des données

In [62]:
df_product.shape

(1050, 15)

In [63]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  1050 non-null   object 
 1   crawl_timestamp          1050 non-null   object 
 2   product_url              1050 non-null   object 
 3   product_name             1050 non-null   object 
 4   product_category_tree    1050 non-null   object 
 5   pid                      1050 non-null   object 
 6   retail_price             1049 non-null   float64
 7   discounted_price         1049 non-null   float64
 8   image                    1050 non-null   object 
 9   is_FK_Advantage_product  1050 non-null   bool   
 10  description              1050 non-null   object 
 11  product_rating           1050 non-null   object 
 12  overall_rating           1050 non-null   object 
 13  brand                    712 non-null    object 
 14  product_specifications  

##### Transformation des types de notre jeu de données

Transformation `crawl_timestamp` en format datetime 

In [64]:
# Transform `crawl_timestamp` to datatime

df_product['crawl_timestamp'] = pd.to_datetime(
    df_product['crawl_timestamp'], format='%Y-%m-%d %H:%M:%S %z')

df_product['crawl_timestamp'] = df_product['crawl_timestamp'].dt.strftime(
    '%d-%m-%Y %H:%M:%S')
# Check if timestamps have no create invalidate date

print(f'Valeur invalide dans `crawl_timestamp` : {
      df_product['crawl_timestamp'].isnull().sum()}')

# Check the value time is correct
df_product['crawl_timestamp'].agg(['min', 'max'])

Valeur invalide dans `crawl_timestamp` : 0


min    01-01-2016 10:55:30
max    31-12-2015 17:07:54
Name: crawl_timestamp, dtype: object

Transformation `retail_price` et `discounted_price` en format numérique 

In [65]:
# Convert dataframe to numeric
df_product['retail_price'] = pd.to_numeric(
    df_product['retail_price'], errors='coerce')
df_product['discounted_price'] = pd.to_numeric(
    df_product['discounted_price'], errors='coerce')
# Check null values on this to columns
df_product.isnull().sum()

uniq_id                      0
crawl_timestamp              0
product_url                  0
product_name                 0
product_category_tree        0
pid                          0
retail_price                 1
discounted_price             1
image                        0
is_FK_Advantage_product      0
description                  0
product_rating               0
overall_rating               0
brand                      338
product_specifications       1
dtype: int64

Transforme `product_specifications` en json exploitable 

In [66]:
# Create an function for apply replacement
def parse_specifications(specs):
    try:
        speecs_corrected = specs.replace('=>', ':')
        return ast.literal_eval(speecs_corrected)
    except:
        return {}


df_product['product_specifications'] = df_product['product_specifications'].apply(
    parse_specifications)

##### Vérification des valeurs manquantes

In [67]:
missing_values = df_product.isnull().sum()
print(missing_values)

uniq_id                      0
crawl_timestamp              0
product_url                  0
product_name                 0
product_category_tree        0
pid                          0
retail_price                 1
discounted_price             1
image                        0
is_FK_Advantage_product      0
description                  0
product_rating               0
overall_rating               0
brand                      338
product_specifications       0
dtype: int64


In [68]:
# Verify the rows of null values
display(df_product[(df_product['retail_price'].isnull())
                   | df_product['discounted_price'].isnull()])

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
2,64d5d4a258243731dc7bbb1eef49ad74,30-04-2016 03:22:56,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{'product_specification': [{'key': 'Material',..."


In [69]:
median_retail_price = df_product['retail_price'].median()
median_discounted_price = df_product['discounted_price'].median()

print(f"Médiane de 'retail_price' : {median_retail_price}")
print(f"Médiane de 'discounted_price' : {median_discounted_price}")
df_product['retail_price'].fillna(median_retail_price, inplace=True)
df_product['discounted_price'].fillna(median_discounted_price, inplace=True)

print(f"Valeurs nulles dans 'retail_price' : {
      df_product['retail_price'].isnull().sum()}")
print(f"Valeurs nulles dans 'discounted_price' : {
      df_product['discounted_price'].isnull().sum()}")

Médiane de 'retail_price' : 999.0
Médiane de 'discounted_price' : 600.0
Valeurs nulles dans 'retail_price' : 0
Valeurs nulles dans 'discounted_price' : 0


/tmp/ipykernel_199048/627864242.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_product['retail_price'].fillna(median_retail_price, inplace=True)
/tmp/ipykernel_199048/627864242.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value

In [70]:
df_product.isnull().sum()

uniq_id                      0
crawl_timestamp              0
product_url                  0
product_name                 0
product_category_tree        0
pid                          0
retail_price                 0
discounted_price             0
image                        0
is_FK_Advantage_product      0
description                  0
product_rating               0
overall_rating               0
brand                      338
product_specifications       0
dtype: int64

In [71]:
df_valeurs_nulles = df_product[df_product.isnull().any(axis=1)]

In [72]:
def extract_brand(description):
    if pd.isnull(description):
        return None
    # Utiliser une expression régulière pour extraire les mots
    words = re.findall(r'\w+', description)

    if not words:
        return None
    # Convertir le premier mot en minuscule pour une comparaison insensible à la casse
    # Vérifier si le premier mot est 'Key' ou 'Specifications'
    if words[0] in 'Buy':
        return words[1]

    elif words[0] == 'Flipkart':
        return words[3]
    else:
        return words[0]


# Appliquer la fonction aux lignes où 'brand' est manquant
df_valeurs_nulles['brand'] = df_valeurs_nulles.apply(
    lambda row: extract_brand(row['description']) if pd.isnull(
        row['brand']) else row['brand'],
    axis=1
)

# Afficher le DataFrame mis à jour
# display(df_valeurs_nulles)

/tmp/ipykernel_199048/4182254686.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valeurs_nulles['brand'] = df_valeurs_nulles.apply(


In [74]:
display(df_valeurs_nulles)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
5,893aa5ed55f7cff2eccea7758d7a86bd,04-12-2015 07:25:36,http://www.flipkart.com/maserati-time-r8851116...,Maserati Time R8851116001 Analog Watch - For ...,"[""Watches >> Wrist Watches >> Maserati Time Wr...",WATEAYAHJ8CUCAYM,24400.0,24400.0,893aa5ed55f7cff2eccea7758d7a86bd.jpg,False,Maserati Time R8851116001 Analog Watch - For ...,No rating available,No rating available,Maserati,{'product_specification': [{'key': 'Chronograp...
6,f355cc1ccb08bd0d283ed979b7ee7515,04-12-2015 07:25:36,http://www.flipkart.com/camerii-wm64-elegance-...,"Camerii WM64 Elegance Analog Watch - For Men,...","[""Watches >> Wrist Watches >> Camerii Wrist Wa...",WATE6Y6D2MZHWGBZ,1099.0,449.0,f355cc1ccb08bd0d283ed979b7ee7515.jpg,False,"Camerii WM64 Elegance Analog Watch - For Men,...",5,5,Camerii,{'product_specification': [{'key': 'Chronograp...
7,dd0e3470a7e6ed76fd69c2da27721041,04-12-2015 07:25:36,http://www.flipkart.com/t-star-uft-tsw-005-bk-...,T STAR UFT-TSW-005-BK-BR Analog Watch - For Boys,"[""Watches >> Wrist Watches >> T STAR Wrist Wat...",WATECFFVGFZPAD3D,999.0,399.0,dd0e3470a7e6ed76fd69c2da27721041.jpg,False,T STAR UFT-TSW-005-BK-BR Analog Watch - For B...,No rating available,No rating available,T,{'product_specification': [{'key': 'Chronograp...
8,41384da51732c0b4df3de8f395714fbb,04-12-2015 07:25:36,http://www.flipkart.com/alfajr-wy16b-youth-dig...,"Alfajr WY16B Youth Digital Watch - For Men, Boys","[""Watches >> Wrist Watches >> Alfajr Wrist Wat...",WATECHFU4QGFQEJG,5495.0,4995.0,41384da51732c0b4df3de8f395714fbb.jpg,False,"Alfajr WY16B Youth Digital Watch - For Men, B...",No rating available,No rating available,Alfajr,{'product_specification': [{'key': 'Chronograp...
9,710ed5f2393a4b9e8823aa0029f71f93,04-12-2015 07:25:36,http://www.flipkart.com/tag-heuer-cau1116-ba08...,TAG Heuer CAU1116.BA0858 Formula 1 Analog Watc...,"[""Watches >> Wrist Watches >> TAG Heuer Wrist ...",WATEA6UGFGKZKDJC,107750.0,107750.0,710ed5f2393a4b9e8823aa0029f71f93.jpg,False,TAG Heuer CAU1116.BA0858 Formula 1 Analog Watc...,No rating available,No rating available,TAG,{'product_specification': [{'key': 'Chronograp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,a618354ba17cff64826558d81afc77ab,01-01-2016 13:15:34,http://www.flipkart.com/elfani-brilliance-lip-...,Elfani Brilliance Lip Color 3.5 g,"[""Beauty and Personal Care >> Makeup >> Lips >...",LSKEDKXZBSVQ883C,255.0,180.0,a618354ba17cff64826558d81afc77ab.jpg,False,Flipkart.com: Buy Elfani Brilliance Lip Color ...,4.5,4.5,Elfani,"{'product_specification': [{'key': 'Quantity',..."
935,7b1c5edc85bc7e31edf49ab70034bb3a,01-01-2016 13:15:34,http://www.flipkart.com/incolor-metalic-lipsti...,Incolor Metalic Lipstick N15 3.8 g,"[""Beauty and Personal Care >> Makeup >> Lips >...",LSKE36A69ZNY64DG,210.0,210.0,7b1c5edc85bc7e31edf49ab70034bb3a.jpg,False,Flipkart.com: Buy Incolor Metalic Lipstick N15...,3,3,Incolor,"{'product_specification': [{'key': 'Quantity',..."
946,2cc24452688ed35dfbff566e22e53c95,01-01-2016 13:15:34,http://www.flipkart.com/organistick-silver-lab...,Organistick Silver Label Lipstick 10 g,"[""Beauty and Personal Care >> Makeup >> Lips >...",LSKE6FWHDDWY7J5T,149.0,149.0,2cc24452688ed35dfbff566e22e53c95.jpg,False,Flipkart.com: Buy Organistick Silver Label Lip...,No rating available,No rating available,Organistick,{'product_specification': [{'key': 'Organic Ty...
985,edb97bd2433fd9de301c8a44344777bb,04-01-2016 03:25:23,http://www.flipkart.com/artist-haat-vanity-box...,Artist Haat Vanity Box Makeup and Jewellery Va...,"[""Beauty and Personal Care >> Makeup >> Vanity...",VANEE2FFWVZZQHJV,1800.0,1499.0,edb97bd2433fd9de301c8a44344777bb.jpg,False,Artist Haat Vanity Box Makeup and Jewellery Va...,No rating available,No rating available,Artist,"{'product_specification': [{'key': 'Weight', '..."
